1. [Loading Data & Libraries](#1)
1. [Data Cleaning & Tokenizing](#2)
1. [Creating Model](#3)
1. [Training Model](#4)
1. [Making Predictions](#5)


<a id = "1"></a><br>
# Loading Data & Libraries

In [1]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.utils import to_categorical
from random import randint
import re
# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg as gut

print(gut.fileids())

[nltk_data] Downloading package gutenberg to /usr/share/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [3]:
macbeth_text = nltk.corpus.gutenberg.raw('shakespeare-macbeth.txt')

In [4]:
print(macbeth_text[:500])

[The Tragedie of Macbeth by William Shakespeare 1603]


Actus Primus. Scoena Prima.

Thunder and Lightning. Enter three Witches.

  1. When shall we three meet againe?
In Thunder, Lightning, or in Raine?
  2. When the Hurley-burley's done,
When the Battaile's lost, and wonne

   3. That will be ere the set of Sunne

   1. Where the place?
  2. Vpon the Heath

   3. There to meet with Macbeth

   1. I come, Gray-Malkin

   All. Padock calls anon: faire is foule, and foule is faire,
Houer through 


<a id = "2"></a><br>
# Data Cleaning & Tokenizing

In [5]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence.lower()

In [6]:
macbeth_text = preprocess_text(macbeth_text)
macbeth_text[:500]

' the tragedie of macbeth by william shakespeare actus primus scoena prima thunder and lightning enter three witches when shall we three meet againe in thunder lightning or in raine when the hurley burley done when the battaile lost and wonne that will be ere the set of sunne where the place vpon the heath there to meet with macbeth come gray malkin all padock calls anon faire is foule and foule is faire houer through the fogge and filthie ayre exeunt scena secunda alarum within enter king malcom'

In [7]:
from nltk.tokenize import word_tokenize

macbeth_text_words = (word_tokenize(macbeth_text))
n_words = len(macbeth_text_words)
unique_words = len(set(macbeth_text_words))

print('Total Words: %d' % n_words)
print('Unique Words: %d' % unique_words)

Total Words: 17250
Unique Words: 3436


In [8]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=3437)
tokenizer.fit_on_texts(macbeth_text_words)

In [9]:
vocab_size = len(tokenizer.word_index) + 1
word_2_index = tokenizer.word_index

In [10]:
print(macbeth_text_words[421])
print(word_2_index[macbeth_text_words[421]])

worthy
158


In [11]:
input_sequence = []
output_words = []
input_seq_length = 100

for i in range(0, n_words - input_seq_length , 1):
    in_seq = macbeth_text_words[i:i + input_seq_length]
    out_seq = macbeth_text_words[i + input_seq_length]
    input_sequence.append([word_2_index[word] for word in in_seq])
    output_words.append(word_2_index[out_seq])

In [12]:
print(input_sequence[0])

[1, 869, 4, 40, 60, 1358, 1359, 408, 1360, 1361, 409, 265, 2, 870, 31, 190, 291, 76, 36, 30, 190, 327, 128, 8, 265, 870, 83, 8, 1362, 76, 1, 1363, 1364, 86, 76, 1, 1365, 354, 2, 871, 5, 34, 14, 168, 1, 292, 4, 649, 77, 1, 220, 41, 1, 872, 53, 3, 327, 12, 40, 52, 1366, 1367, 25, 1368, 873, 328, 355, 9, 410, 2, 410, 9, 355, 1369, 356, 1, 1370, 2, 874, 169, 103, 127, 411, 357, 149, 31, 51, 1371, 329, 107, 12, 358, 412, 875, 1372, 51, 20, 170, 92, 9]


In [13]:
X = np.reshape(input_sequence, (len(input_sequence), input_seq_length, 1))
X = X / float(vocab_size)

y = to_categorical(output_words)

In [14]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (17150, 100, 1)
y shape: (17150, 3437)


<a id = "3"></a><br>
# Creating Model

In [15]:
model = Sequential()
model.add(LSTM(40, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(40, return_sequences=True))
model.add(LSTM(40))
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 40)           6720      
                                                                 
 lstm_1 (LSTM)               (None, 100, 40)           12960     
                                                                 
 lstm_2 (LSTM)               (None, 40)                12960     
                                                                 
 dense (Dense)               (None, 3437)              140917    
                                                                 
Total params: 173,557
Trainable params: 173,557
Non-trainable params: 0
_________________________________________________________________


<a id = "4"></a><br>
# Training Model

In [16]:
model.fit(X, y, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
268/268 [==============================] - 37s 119ms/step - loss: 6.9993
Epoch 2/10
268/268 [==============================] - 32s 119ms/step - loss: 6.6383
Epoch 3/10
268/268 [==============================] - 32s 119ms/step - loss: 6.6238
Epoch 4/10
268/268 [==============================] - 31s 116ms/step - loss: 6.6179
Epoch 5/10
268/268 [==============================] - 31s 116ms/step - loss: 6.6157
Epoch 6/10
268/268 [==============================] - 31s 117ms/step - loss: 6.6139
Epoch 7/10
268/268 [==============================] - 31s 117ms/step - loss: 6.6123
Epoch 8/10
268/268 [==============================] - 31s 116ms/step - loss: 6.6097
Epoch 9/10
268/268 [==============================] - 32s 118ms/step - loss: 6.6079
Epoch 10/10
268/268 [==============================] - 32s 118ms/step - loss: 6.6053


<a id = "5"></a><br>
# Making Predictions

In [17]:
random_seq_index = np.random.randint(0, len(input_sequence)-1)
random_seq = input_sequence[random_seq_index]

index_2_word = dict(map(reversed, word_2_index.items()))

word_sequence = [index_2_word[value] for value in random_seq]

print(' '.join(word_sequence))

of horrid hell can come diuell more damn in euils to top macbeth mal grant him bloody luxurious auaricious false deceitfull sodaine malicious smacking of euery sinne that ha a name but there no bottome none in my voluptuousnesse your wiues your daughters your matrons and your maides could not fill vp the cesterne of my lust and my desire all continent impediments would ore beare that did oppose my will better macbeth then such an one to reigne macd boundlesse intemperance in nature is tyranny it hath beene th vntimely emptying of the happy throne and fall of many


In [18]:
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))
    int_sample = int_sample / float(vocab_size)

    predicted_word_index = model.predict(int_sample, verbose=0)

    predicted_word_id = np.argmax(predicted_word_index)
    seq_in = [index_2_word[index] for index in random_seq]

    word_sequence.append(index_2_word[ predicted_word_id])

    random_seq.append(predicted_word_id)
    random_seq = random_seq[1:len(random_seq)]

In [19]:
final_output = ""
for word in word_sequence:
    final_output = final_output + " " + word

print(final_output)

 of horrid hell can come diuell more damn in euils to top macbeth mal grant him bloody luxurious auaricious false deceitfull sodaine malicious smacking of euery sinne that ha a name but there no bottome none in my voluptuousnesse your wiues your daughters your matrons and your maides could not fill vp the cesterne of my lust and my desire all continent impediments would ore beare that did oppose my will better macbeth then such an one to reigne macd boundlesse intemperance in nature is tyranny it hath beene th vntimely emptying of the happy throne and fall of many the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
